<a href="https://colab.research.google.com/github/stellagerantoni/LatentCfMultivariate/blob/main/FingerMovements_simple_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/stellagerantoni/LatentCfMultivariate

In [ ]:
!pip install -q wildboar
!pip install -q scikit-learn
!pip install -q stumpy
!pip install -q fastdtw
!pip install aeon

In [ ]:
import logging
import os
import warnings
from argparse import ArgumentParser
from aeon.datasets import load_classification

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.spatial import distance_matrix
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from wildboar.datasets import load_dataset
from wildboar.ensemble import ShapeletForestClassifier
from wildboar.explain.counterfactual import counterfactuals
%cd '/content/LatentCfMultivariate'
from _guided import ModifiedLatentCF
from help_functions import *
from keras_models import *

In [4]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
RANDOM_STATE = 39

## **FUNCTIONS**

In [5]:
def load_dataset(dataset):
  X, y = load_classification(dataset)
  if dataset == 'FingerMovements':
    pos = 'left'
    neg = 'right'


  print(" Shape of X = ", X.shape)
  print(" Shape of y = ", y.shape)
  #print(" Meta data = ", meta_data)
  # Convert positive and negative labels to 1 and 0
  pos_label, neg_label = 1, 0
  if pos != pos_label:
      y[y==pos] = pos_label # convert/normalize positive label to 1
  if neg != neg_label:
      y[y==neg] = neg_label # convert negative label to 0

  y = y.astype(int)
  print(f"\n X[:1] = \n{X[:1]}")
  return X,y,pos_label, neg_label

# **ACTUALL CODE**
datasets available : 'Heartbeat', 'SelfRegulationSCP1'

In [6]:
RANDOM_STATE = 39
X,y,pos_label,neg_label = load_dataset('FingerMovements')
X = X.transpose(0,2,1)
print(f'shape of X = {X.shape}')
print(f'shape of y = {y.shape}')
#print(f'data imformation = {data_information}')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(f'shape of X train = {X_train.shape}')
print(f'shape of y train = {y_train.shape}')

 Shape of X =  (416, 28, 50)
 Shape of y =  (416,)

 X[:1] = 
[[[41.8 44.8 47.1 ... 69.8 72.6 76.1]
  [55.2 53.8 59.9 ... 17.5 28.  12.1]
  [-8.6 -3.6 14.4 ... 23.3 35.9 23.2]
  ...
  [16.9 24.5 24.5 ... 51.9 59.6 57.3]
  [42.2 35.  41.7 ... 51.5 58.5 46.9]
  [13.  26.6 52.5 ... -3.5 -3.2 -2.6]]]
shape of X = (416, 50, 28)
shape of y = (416,)
shape of X train = (332, 50, 28)
shape of y train = (332,)


In [7]:
#Upsample the minority class

unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'before: {class_counts}')
X_train,y_train = upsample_minority_multivariate(X_train,y_train)
X,y = upsample_minority_multivariate(X, y)
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'after: {class_counts}')

before: [166 166]
after: [166 166]


In [8]:
#Processing and Padding all our data
#Padding needed for autoencoder

n_training,n_timesteps, n_features= X_train.shape

X, trained_scaler =  normalize_multivariate(data=X, n_timesteps=n_timesteps, n_features = n_features)
X_train_processed, trained_scaler =  normalize_multivariate(data=X_train, n_timesteps=n_timesteps, n_features = n_features)
X_test_processed, _ =  normalize_multivariate(data=X_test, n_timesteps=n_timesteps, scaler=trained_scaler, n_features = n_features)

X, padding_size = conditional_pad_multivariate(X)
X_train_processed_padded, padding_size = conditional_pad_multivariate(X_train_processed) # add extra padding zeros if n_timesteps cannot be divided by 4, required for 1dCNN autoencoder structure
X_test_processed_padded, _ = conditional_pad_multivariate(X_test_processed)

n_timesteps_padded = X_train_processed_padded.shape[1]
print(f"Data pre-processed, original #timesteps={n_timesteps}, padded #timesteps={n_timesteps_padded}.")

#check the processing (0,1) min should be min 0 and max should be max 1
print(f"\nmin value = {np.min(X_train)}, max value = {np.max(X_train)}")
print(f"min value normalized = {np.min(X_train_processed)}, max value normalized= {np.max(X_train_processed)}")

#check that padding paddes the right dimention
print(f"\nX_train.shape = {X_train.shape}" )
print(f"X_train_processed_padded.shape = {X_train_processed_padded.shape}")


Data pre-processed, original #timesteps=50, padded #timesteps=52.

min value = -132.1, max value = 205.1
min value normalized = 0.0, max value normalized= 1.0

X_train.shape = (332, 50, 28)
X_train_processed_padded.shape = (332, 52, 28)


In [9]:
#splitting the dataset

from sklearn.model_selection import train_test_split
X_train,X_validation, y_train, y_validation = train_test_split(X_train_processed_padded, y_train, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train)

In [10]:
#Getting the two forms of labels needed
#-the y_classes (1,0,1,0,...)
#-the y (one hot encoded)

print(f'X_train = {X_train.shape}')
print(f'X_validation = {X_validation.shape}')
print(f'X_test = {X_test.shape}')

y_classes = y
y_train_classes = y_train
y_validation_classes = y_validation
y_test_classes = y_test

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, len(np.unique(y)))
y_train = to_categorical(y_train, len(np.unique(y_train)))
y_validation = to_categorical(y_validation, len(np.unique(y_validation)))
y_test = to_categorical(y_test, len(np.unique(y_test)))

print(f'\ny_train_classes = {y_train_classes.shape}, y_validation_classes = {y_validation_classes.shape}, y_test_classes = {y_test_classes.shape}')
print(f'y_train = {y_train.shape}, y_validation = {y_validation.shape}, y_test= {y_test.shape}')

X_train = (265, 52, 28)
X_validation = (67, 52, 28)
X_test = (84, 50, 28)

y_train_classes = (265,), y_validation_classes = (67,), y_test_classes = (84,)
y_train = (265, 2), y_validation = (67, 2), y_test= (84, 2)


In [11]:
def ClassifierLSTM(n_timesteps, n_features, extra_lstm_layer=True, n_output=1):
    # Define the model structure - only LSTM layers
    # https://www.kaggle.com/szaitseff/classification-of-time-series-with-lstm-rnn
    inputs = keras.Input(shape=(n_timesteps, n_features), dtype="float32")
    if extra_lstm_layer:
        x = keras.layers.LSTM(64, activation="tanh", return_sequences=True)(
            inputs
        )  # set return_sequences true to feed next LSTM layer
    else:
        x = keras.layers.LSTM(32, activation="tanh", return_sequences=False)(
            inputs
        )  # set return_sequences false to feed dense layer directly
    x = keras.layers.BatchNormalization()(x)
    # x = keras.layers.LSTM(32, activation='tanh', return_sequences=True)(x)
    # x = keras.layers.BatchNormalization()(x)
    if extra_lstm_layer:
        x = keras.layers.LSTM(16, activation="tanh", return_sequences=False)(x)
        x = keras.layers.BatchNormalization()(x)

    if n_output >= 2:
        outputs = keras.layers.Dense(n_output, activation="softmax")(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid")(x)

    classifier2 = keras.Model(inputs, outputs)

    return classifier2

In [12]:


# ## LatentCF++ models
# reset seeds for numpy, tensorflow, python random package and python environment seed
reset_seeds()
###############################################
# ### LSTM classifier

classifier = ClassifierLSTM(
    n_timesteps_padded, n_features, n_output=2
)

optimizer = keras.optimizers.Adam(lr=0.001)
classifier.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
)

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=15, restore_best_weights=True
)
# Train the model
reset_seeds()
print("Training log for LSTM-FCN classifier:")
classifier_history = classifier.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=12,
    shuffle=True,
    verbose=True,
    validation_data=(X_validation, y_validation),
    callbacks=[early_stopping_accuracy],
)

y_pred = classifier.predict(X_test_processed_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
acc = balanced_accuracy_score(y_true=y_test_classes, y_pred=y_pred_classes)
print(f"LSTM-FCN classifier trained, with validation accuracy {acc}.")

confusion_matrix_df = pd.DataFrame(
    confusion_matrix(y_true=y_test_classes, y_pred=y_pred_classes, labels=[1, 0]),
    index=["True:1", "True:0"],
    columns=["Pred:1", "Pred:0"],
)
print(confusion_matrix_df)


Training log for LSTM-FCN classifier:
Epoch 1/150
23/23 [==============================] - 11s 107ms/step - loss: 0.7501 - accuracy: 0.5509 - val_loss: 0.6920 - val_accuracy: 0.4925
Epoch 2/150
23/23 [==============================] - 0s 20ms/step - loss: 0.6782 - accuracy: 0.6038 - val_loss: 0.6893 - val_accuracy: 0.5224
Epoch 3/150
23/23 [==============================] - 1s 25ms/step - loss: 0.6500 - accuracy: 0.6151 - val_loss: 0.6923 - val_accuracy: 0.4925
Epoch 4/150
23/23 [==============================] - 1s 32ms/step - loss: 0.6365 - accuracy: 0.6151 - val_loss: 0.6883 - val_accuracy: 0.6866
Epoch 5/150
23/23 [==============================] - 1s 30ms/step - loss: 0.5749 - accuracy: 0.7396 - val_loss: 0.6800 - val_accuracy: 0.5373
Epoch 6/150
23/23 [==============================] - 1s 29ms/step - loss: 0.5768 - accuracy: 0.7057 - val_loss: 0.6937 - val_accuracy: 0.5373
Epoch 7/150
23/23 [==============================] - 1s 26ms/step - loss: 0.5524 - accuracy: 0.7434 - val_lo

In [13]:
def AutoencoderLSTM(n_timesteps, n_features):
    # Define encoder and decoder structure
    # structure from medium post: https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352
    def EncoderLSTM(input):
        # x = keras.layers.LSTM(64, activation='relu', return_sequences=True)(input)
        x = keras.layers.LSTM(64, activation="tanh", return_sequences=True)(input)
        # encoded = keras.layers.LSTM(32, activation='relu', return_sequences=False)(x)
        encoded = keras.layers.LSTM(32, activation="tanh", return_sequences=False)(x)
        return encoded

    def DecoderLSTM(encoded):
        x = keras.layers.RepeatVector(n_timesteps)(encoded)
        # x = keras.layers.LSTM(32, activation='relu', return_sequences=True)(x)
        x = keras.layers.LSTM(32, activation="tanh", return_sequences=True)(x)
        # x = keras.layers.LSTM(64, activation='relu', return_sequences=True)(x)
        x = keras.layers.LSTM(64, activation="tanh", return_sequences=True)(x)
        decoded = keras.layers.TimeDistributed(
            keras.layers.Dense(n_features, activation="sigmoid")
        )(x)
        return decoded

    # Define the AE model
    orig_input2 = keras.Input(shape=(n_timesteps, n_features))

    autoencoder2 = keras.Model(
        inputs=orig_input2, outputs=DecoderLSTM(EncoderLSTM(orig_input2))
    )

    return autoencoder2

In [14]:
reset_seeds()

# ### LSTM autoencoder
autoencoder = AutoencoderLSTM( n_timesteps_padded,n_features)
optimizer = keras.optimizers.Adam(lr=0.0005)
autoencoder.compile(optimizer=optimizer, loss="mse")

# Define the early stopping criteria
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, restore_best_weights=True)
# Train the model
reset_seeds()
print("Training log for 1dCNN autoencoder:")
autoencoder_history = autoencoder.fit(
    X_train,
    X_train,
    epochs=50,
    batch_size=12,
    shuffle=True,
    verbose=2,
    validation_data=(X_validation, X_validation),
    callbacks=[early_stopping])

ae_val_loss = np.min(autoencoder_history.history['val_loss'])
print(f"1dCNN autoencoder trained, with validation loss: {ae_val_loss}.")


Training log for 1dCNN autoencoder:
Epoch 1/50
23/23 - 8s - loss: 0.0295 - val_loss: 0.0232 - 8s/epoch - 357ms/step
Epoch 2/50
23/23 - 0s - loss: 0.0227 - val_loss: 0.0196 - 324ms/epoch - 14ms/step
Epoch 3/50
23/23 - 0s - loss: 0.0189 - val_loss: 0.0180 - 331ms/epoch - 14ms/step
Epoch 4/50
23/23 - 0s - loss: 0.0174 - val_loss: 0.0163 - 330ms/epoch - 14ms/step
Epoch 5/50
23/23 - 0s - loss: 0.0163 - val_loss: 0.0149 - 431ms/epoch - 19ms/step
Epoch 6/50
23/23 - 0s - loss: 0.0148 - val_loss: 0.0139 - 476ms/epoch - 21ms/step
Epoch 7/50
23/23 - 0s - loss: 0.0131 - val_loss: 0.0143 - 446ms/epoch - 19ms/step
Epoch 8/50
23/23 - 0s - loss: 0.0125 - val_loss: 0.0133 - 467ms/epoch - 20ms/step
Epoch 9/50
23/23 - 0s - loss: 0.0127 - val_loss: 0.0128 - 432ms/epoch - 19ms/step
Epoch 10/50
23/23 - 0s - loss: 0.0114 - val_loss: 0.0096 - 316ms/epoch - 14ms/step
Epoch 11/50
23/23 - 0s - loss: 0.0097 - val_loss: 0.0092 - 327ms/epoch - 14ms/step
Epoch 12/50
23/23 - 0s - loss: 0.0090 - val_loss: 0.0127 - 340

In [ ]:
#Gettting the Global weights, needed for counterfactuals

from _guided import get_global_weights
from help_functions import evaluate
pos_label = 1
neg_label = 0

step_weights = get_global_weights(
        X,
        y_classes,
        classifier,
        n_timesteps= n_timesteps,
        n_features=n_features,
        random_state=RANDOM_STATE,
)


In [16]:
step_weights

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]])

In [17]:
reset_seeds()
cf_model = ModifiedLatentCF(
    probability=0.5,tolerance=1e-6, max_iter=500, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),autoencoder = autoencoder,
    pred_margin_weight=0.7, step_weights = step_weights, random_state= RANDOM_STATE)
cf_model.fit(classifier)

y_neg = y_classes[y_classes == 0]
X_neg = X[y_classes == 0]


with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning) # ignore warning of matrix multiplication: https://stackoverflow.com/questions/29688168/mean-nanmean-and-warning-mean-of-empty-slice
    cf_embeddings, losses, weights = cf_model.transform(X_neg, y_neg) #self, x, pred_label
cf_pred_labels = classifier.predict(cf_embeddings)[:,1]# predicted probabilities of CFs
for idx in range(cf_pred_labels.shape[0]):
  if cf_pred_labels[idx] > 0.5:
    cf_pred_labels[idx] = 1
  else:
    cf_pred_labels[idx] = 0

print(f'Transformation_finished with validity_score = {validity_score(y_neg,cf_pred_labels)}')

1 samples been transformed.
26 samples been transformed.
51 samples been transformed.
76 samples been transformed.
101 samples been transformed.
126 samples been transformed.
151 samples been transformed.
176 samples been transformed.
201 samples been transformed.
208 samples been transformed, in total.
7/7 [==============================] - 0s 6ms/step
Transformation_finished with validity_score = 0.16346153846153846


In [37]:
#Validity
cf_pred = classifier.predict(cf_embeddings)[:, 1] # predicted probabilities of CFs
cf_pred_labels = cf_pred
for idx in range(cf_pred_labels.shape[0]):
  if cf_pred_labels[idx] > 0.5:
    cf_pred_labels[idx] = 1
  else:
    cf_pred_labels[idx] = 0


print(f'Transformation_finished with validity_score = {validity_score(y_neg,cf_pred_labels)}')

7/7 [==============================] - 1s 5ms/step
Transformation_finished with validity_score = 0.0


In [ ]:
#Calculating MAE
from tensorflow.keras.losses import MeanSquaredError
total = 0
probability = 0.5
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = classifier.predict(counterfactual)[:, 1]
    dist = (prediction - probability)
    total +=dist
mean_mse = total /cf_embeddings.shape[0]


In [19]:
print(f"The Mean MSE of the data is: {mean_mse} ")

The Mean MSE of the data is: [-0.00422032] 


In [ ]:
#Calculating absolute MAE
from tensorflow.keras.losses import MeanSquaredError
total = 0
probability = 0.5
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = classifier.predict(counterfactual)[:, 1]
    dist = abs(prediction - probability)
    total +=dist
mean_mse = total /cf_embeddings.shape[0]


In [21]:
print(f"The Absolute Mean MSE of the data is: {mean_mse} ")

The Absolute Mean MSE of the data is: [0.00437415] 


In [22]:
def remove_paddings(cf_samples, padding_size):
    if padding_size != 0:
        # use np.squeeze() to cut the last time-series dimension, for evaluation
        cf_samples = np.squeeze(cf_samples[:, :-padding_size, :])
    else:
        cf_samples = np.squeeze(cf_samples)
    return cf_samples

In [23]:
X = remove_paddings(X, padding_size)
cf_embeddings = remove_paddings(cf_embeddings, padding_size)


In [29]:
X_neg = remove_paddings(X_neg, padding_size)

In [30]:
#Proximity
def euclidean_distance(X, cf_samples):
    paired_distances = np.linalg.norm(X - cf_embeddings, axis=1)
    return np.mean(paired_distances)
euclidean_distance(X_neg, cf_embeddings)

0.4627038364525892

In [31]:
from scipy.stats import gaussian_kde
diffrences_from_abnormal = []
diffrences_from_normal = []
for dimention in range(cf_embeddings.shape[2]):


  abnormal_data = X[y_classes == 1][:,:,dimention]
  normal_data = X[y_classes == 0][:,:,dimention]
  counterf_data = cf_embeddings[:,:,dimention]

  #get the kernel for every dimention of the trained
  kernel = gaussian_kde(abnormal_data.T,bw_method=None)

  #get all the log likelihoods
  log_likelihood_abnormal = np.mean(kernel.logpdf(abnormal_data.T))
  log_likelihood_normal = np.mean(kernel.logpdf(normal_data.T))
  log_likelihood_counterfactual = np.mean(kernel.logpdf(counterf_data.T))

  #get the diffrences from the counterfactuals
  diff_from_abnormal = abs(log_likelihood_counterfactual-log_likelihood_abnormal)
  diffrences_from_abnormal.append(diff_from_abnormal)

  diff_from_normal = abs(log_likelihood_counterfactual-log_likelihood_normal)
  diffrences_from_normal.append(diff_from_normal)



In [32]:
print(diffrences_from_normal)

[1.319245823898342, 30.455479315366645, 11.456162350348322, 91.36786817429801, 27.56239619988365, 5.385884867198627, 0.7903023751492242, 7.8606076803686875, 2.9591112727882347, 15.125670955078775, 4.080598790863817, 3.544469534831464, 3.764394201063439, 125.9247666734058, 108.9011087470796, 24.263203703755067, 50.21031744895313, 52.34357761870318, 156.92961111230878, 60.921815623780724, 2.815689979201622, 17.676704173220983, 1.1843042221529316, 22.812922552669022, 64.68918541843254, 11.811773632068949, 32.80770726394123, 18.08795221502851]


In [33]:
print(diffrences_from_abnormal)

[55.67406324728471, 84.02257313177826, 68.42957134727155, 148.60510984693525, 84.91131115644293, 61.71372584336784, 58.23127427529269, 65.46191161632603, 59.120155494043544, 38.47776560357828, 50.13569710818449, 50.585340888776315, 50.295011201034015, 179.78201586728636, 162.64330996899184, 78.95046647101411, 105.32995502579502, 110.60386447951586, 217.94980176755206, 123.26246806653302, 61.69396315023208, 69.46853274352074, 55.01226249990762, 74.24292680503808, 116.76048356132861, 64.41838615969382, 84.76028191606343, 71.47377625842687]


In [34]:
print(np.mean(diffrences_from_normal))

34.18045828306569


In [35]:
print(np.mean(diffrences_from_abnormal))

87.57200019647199
